In [5]:
from bhpwave.spline import CubicSpline, BicubicSpline
from bhpwave.trajectory.geodesic import kerr_isco_frequency, kerr_circ_geo_radius

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rc('text', usetex=True)
mpl.rc('font', **{'size' : 14})

import os
pathname=os.path.abspath("")

OMEGA_MIN = 2.e-3
A_MAX = 0.9999

def alpha_of_a_omega(a, omega):
    oISCO = kerr_isco_frequency(a)
    return alpha_of_omega_ISCO(omega, oISCO)

def alpha_of_omega_ISCO(omega, oISCO):
    return (abs(oISCO**(1./3.) - omega**(1./3.))/(oISCO**(1./3.) - OMEGA_MIN**(1./3.)))**(0.5)

def omega_of_a_alpha(a, alpha):
    oISCO = kerr_isco_frequency(a)
    return omega_of_alpha_ISCO(alpha, oISCO)

def omega_of_alpha_ISCO(alpha, oISCO):
    return pow(pow(oISCO, 1./3.) - pow(alpha, 2.)*(pow(oISCO, 1./3.) - pow(OMEGA_MIN, 1./3.)), 3.)

def chi_of_spin_subfunc(a):
    return pow(1. - a, 1./3.)

def chi_of_spin(a):
    return pow((chi_of_spin_subfunc(a) - chi_of_spin_subfunc(A_MAX))/(chi_of_spin_subfunc(-A_MAX) - chi_of_spin_subfunc(A_MAX)), 0.5)

def spin_of_chi(chi):
    return 1. - pow(chi_of_spin_subfunc(A_MAX) + pow(chi, 2.)*(chi_of_spin_subfunc(-A_MAX) - chi_of_spin_subfunc(A_MAX)), 3.)

def a_omega_to_chi_alpha(a, omega):
    chi = chi_of_spin(a)
    alpha = alpha_of_a_omega(a, omega)
    return (chi, alpha)

def pn_flux_noprefactor(omega):
    return omega**(10./3.)

def pn_time_noprefactor(a, omega):
    oISCO = kerr_isco_frequency(a)
    offset = oISCO**(-8/3) + 1.e-6
    return -np.abs(offset - omega**(-8./3.))

def pn_phase_noprefactor(a, omega):
    oISCO = kerr_isco_frequency(a)
    offset = oISCO**(-5/3) + 1.e-6
    return -np.abs(offset - omega**(-5./3.))

def pn_flux_noprefactor_domega(omega):
    return (10./3.)*omega**(7./3.)

def pn_time_noprefactor_domega(omega):
    return -(-8/3)*omega**(-11./3.)

def pn_phase_noprefactor_domega(omega):
    return -(-5/3)*omega**(-8./3.)

def load_trajectory_data_file(filepath):
    traj = np.loadtxt(filepath, skiprows=3)
    #trajHeader = np.loadtxt(filepath, skiprows=2, max_rows=1, dtype='str')
    trajShape = np.loadtxt(filepath, skiprows=1, max_rows=1, dtype='int')

    fluxDataTemp = np.ascontiguousarray(traj[:, 2].reshape(trajShape[:2]))
    alphaDataFluxTemp = np.ascontiguousarray(traj[:, 1].reshape(trajShape[:2])[0])
    chiDataFluxTemp = np.ascontiguousarray(traj[:, 0].reshape(trajShape[:2])[:, 0])

    phaseData = np.ascontiguousarray(traj[:, 4].reshape(trajShape[:2]))
    timeData = np.ascontiguousarray(traj[:, 3].reshape(trajShape[:2]))
    alphaData = np.ascontiguousarray(traj[:, 1].reshape(trajShape[:2])[0])
    chiData = np.ascontiguousarray(traj[:, 0].reshape(trajShape[:2])[:, 0])
    betaData = np.ascontiguousarray(traj[:, 5].reshape(trajShape[:2])[0])
    omegaData = np.ascontiguousarray(traj[:, 6].reshape(trajShape[:2]))
    phaseBetaData = np.ascontiguousarray(traj[:, 7].reshape(trajShape[:2]))

    fluxDownsampleChi = int((trajShape[0] - 1)/(trajShape[2] - 1))
    fluxDownsampleAlpha = int((trajShape[1] - 1)/(trajShape[3] - 1))

    fluxData = fluxDataTemp[::fluxDownsampleChi, ::fluxDownsampleAlpha]
    alphaDataFlux = alphaDataFluxTemp[::fluxDownsampleAlpha]
    chiDataFlux = chiDataFluxTemp[::fluxDownsampleChi]

    return chiData, alphaData, timeData, phaseData, betaData, omegaData, phaseBetaData, chiDataFlux, alphaDataFlux, fluxData

traj_data_full = load_trajectory_data_file(pathname+"/../data/trajectory.txt")

timeData_temp = traj_data_full[2]
phaseData_temp = traj_data_full[3]

chiData = traj_data_full[7]
alphaData = traj_data_full[8]
fluxData = traj_data_full[9]

downsample_chi = int((phaseData_temp.shape[0] - 1)/(fluxData.shape[0] - 1))
downsample_alpha = int((phaseData_temp.shape[1] - 1)/(fluxData.shape[1] - 1))
phaseData = phaseData_temp[::downsample_chi, ::downsample_alpha]
timeData = timeData_temp[::downsample_chi, ::downsample_alpha]

Edot = BicubicSpline(chiData, alphaData, fluxData)
PhiCheck = BicubicSpline(chiData, alphaData, phaseData)
TCheck = BicubicSpline(chiData, alphaData, timeData)

downsample_rate = 4
Nb = int((chiData.shape[0] - 1)/downsample_rate + 1)
Na = int((alphaData.shape[0] - 1)/downsample_rate + 1)
flux_samples = np.zeros((Nb, Na, 5))
phase_samples = np.zeros((Nb, Na, 5))
time_samples = np.zeros((Nb, Na, 5))
for i in range(Nb):
    for j in range(Na):
        chi = chiData[downsample_rate*i]
        alpha = alphaData[downsample_rate*j]
        atemp = spin_of_chi(chi)
        otemp = omega_of_a_alpha(atemp, alpha)
        EdotData = Edot(chi, alpha)
        PData = PhiCheck(chi, alpha)
        TData = TCheck(chi, alpha)
        flux_samples[i, j] = [atemp, otemp, EdotData, alpha, chi]
        phase_samples[i, j] = [atemp, otemp, PData, alpha, chi]
        time_samples[i, j] = [atemp, otemp, TData, alpha, chi]

In [7]:
time_samples[20,20]

array([ 0.85904034,  0.15743033, -0.58495866,  0.3125    ,  0.625     ])

In [10]:
pn_time_noprefactor(0.9411845318952371, 0.16569872100996472)

85.59816824968382

In [13]:
kerr_isco_frequency(0.9411845318952371)**(-5/3) - (0.16569872100996472)**(-5/3)

-10.75648372334578